Please take advantage of the two extra hours in your schedule from not having lectures on Wednesday and Friday this week to make substantial progress on your project.  For Monday, November 4, please list what you proposed to have completed by the end of Week 5, and summarize what you have accomplished.  You can be as detailed as you like.  This counts towards your preliminary report score.

# Proposed to be done by the end of the week:
*  Finish the frequency analysis of live music clips.   
    * right now I have done frequency analysis for 1 live music,I need to finish the rest 4 live music.
    * also right now my function that seperates each music from the large audio file does not seem to be working, so I need to fix that function so that I can process each music separatly.



# What Have I done so far:

## Data gathering and cleaning

### Importing a noise dataset from kaggle

In [ ]:
import kagglehub
path = kagglehub.dataset_download("minsithu/audio-noise-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.54M/1.54M [00:00<00:00, 13.2MB/s]

Extracting files...
Path to dataset files: /Users/primepi/.cache/kagglehub/datasets/minsithu/audio-noise-dataset/versions/1


### Getting audio file separated from live music video file for audio processing.

In [ ]:
from moviepy.editor import VideoFileClip


video_file = 'input_video.mov'
audio_output = 'output_audio.wav'

# Load the video file
video_clip = VideoFileClip(video_file)

# Extract audio
audio_clip = video_clip.audio

# Write the audio to a WAV file
audio_clip.write_audiofile(audio_output)


MoviePy - Writing audio in output_audio.wav


MoviePy - Done.


### Separating the one large audio file to mutiple live music songs 

In [1]:
from pydub import AudioSegment
from pydub.silence import split_on_silence

# Load the audio file
audio_file = 'output_audio.wav'
audio = AudioSegment.from_wav(audio_file)

# Parameters for silence detection
min_silence_len = 2000  # Minimum length of silence (in ms) to be used for a split
silence_thresh = -50    # Silence threshold (in dBFS)

# Split audio where silence is 'silent' for at least 'min_silence_len' milliseconds
chunks = split_on_silence(
    audio,
    min_silence_len=min_silence_len,
    silence_thresh=silence_thresh
)

# Export each chunk as a separate audio file
for i, chunk in enumerate(chunks):
    chunk_file = f"song_{i+1}.wav"
    print(f"Exporting {chunk_file}")
    chunk.export(chunk_file, format="wav")


Exporting song_1.wav


### Fourier transform on each song for frequency Analysis

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import glob
import os

# Get a list of all song files
song_files = glob.glob('song_*.mp3')

for song_file in song_files:
    # Read the WAV file
    sample_rate, data = wavfile.read(song_file)
    
    # If stereo, take only one channel
    if len(data.shape) > 1:
        data = data[:, 0]

    # Normalize data
    data = data / np.max(np.abs(data))

    # Number of samples
    N = len(data)

    # Compute Fourier Transform
    yf = np.fft.fft(data)
    xf = np.fft.fftfreq(N, 1 / sample_rate)

    # Only take the positive frequencies
    idxs = np.where(xf >= 0)
    xf = xf[idxs]
    yf = np.abs(yf[idxs])

    # Plot the frequency spectrum
    plt.figure(figsize=(12, 6))
    plt.plot(xf, yf)
    plt.title(f'Frequency Spectrum of {os.path.basename(song_file)}')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')
    plt.grid(True)
    plt.show()


ValueError: File format b'ID3\x02' not understood. Only 'RIFF' and 'RIFX' supported.

## Next Step
* Gather clean studio record music of the same song for each live music.
* Fourier transform them and get their frequency plot.
* Time analysis to align each live music with their clean studio record.
* Apply polynomial function that derived from time and frequency analysis of the song to denoise the live music while maintaining its natural form.